#### Problem Statement 3: Identify key patterns observed in the dataset, highlighting significant trends or correlations that provide valuable insights. At least 5 major findings

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
Activity_Details = pd.read_excel("Activity Details.xlsx")
print("Activity Details")
print(Activity_Details.head())

Advisors_Details = pd.read_excel("Advisors Details.xlsx")
print("\nAdvisors Details")
print(Advisors_Details.head())

Asset_Details = pd.read_excel("Asset Details.xlsx")
print("\nAsset Details")
print(Asset_Details.head())

Firm_Details = pd.read_excel("Firm Details.xlsx")
print("\nFirm Details")
print(Firm_Details.head())

Transaction_Details = pd.read_excel("Transactional Details.xlsx")
print("\nTransactional Details")
print(Transaction_Details.head())

Activity Details
   CONTACT_ID ACTIVITY TYPE ACTIVITY DATE  ACTIVITY DATE KEY
0        1153     PhoneCall    2022-06-02           20220602
1        1129          Note    2022-06-22           20220622
2        1700          Note    2022-07-21           20220721
3        1978         Email    2021-09-28           20210928
4        1978      FollowUp    2022-03-16           20220316

Advisors Details
   CONTACT ID NICK_NAME FIRST_NAME MIDDLE_NAME LAST_NAME PREFIX SUFFIX  \
0        1001       Tua       Tuan         NaN  Roettgen    NaN    NaN   
1        1002       Cha       Chad         NaN  Stofleth    Ms.    NaN   
2        1003       Arl      Arlen         NaN  Bierling    NaN    NaN   
3        1004       Oti       Otis         NaN    Mathen    NaN    NaN   
4        1005       Sun       Sung         NaN      Kiep    Mr.    NaN   

   MOBILE_PHONE  FIRM ID              PRIMARY_EMAIL UNSUBSCRIBED  \
0  554-882-3214    12087   Tuan.Roettgen@Janney.com  Not Blocked   
1  215-781-4188   

In [3]:
Asset_Details = Asset_Details.sort_values(['ASSET_MONTH_KEY', 'CONTACT_ID'])

The metric we are going to use to find the advisors with significant drop in the assets is by doing the following:<br>
* Group the Asset_Details dataset by the contact_id's of the advisors.
* Find the mean of the Asset Price for first half of the months. Then for the second half of the months. Then find the percentage change between the two.
* For this problem statement, I considered atleast 50% deprecation to be significant.

In [4]:
def fun(serobj):
    l = serobj.size
    if l==1: return 0
    m1 = serobj[0:l//2].mean()
    m2 = serobj[l//2:l].mean()
    return ((m1-m2)*100)/m1

In [5]:
df = Asset_Details.groupby('CONTACT_ID')['USD_MV'].apply(fun).sort_values()
df = df.reset_index()

In [6]:
df = df.rename(columns={"USD_MV":"Percent Change"})

In [7]:
target = df[df['Percent Change']<0]
target

,CONTACT_ID,Percent Change
0,1399,-4235.966856
1,1193,-3872.147523
2,1283,-3684.146514
3,1270,-3348.831577
4,1583,-3319.840957
...,...,...
565,1389,-2.594722
566,1368,-1.965583
567,1254,-1.640222
568,1727,-1.533517


Now we also need to find the non-markable advisors from the list of advisors. The Criteria include:
* Those advisors who do not have any email ID.
* Those advisors who were never contacted through email by the Sales Team.

In [14]:
cond1 = Advisors_Details[pd.notna(Advisors_Details['PRIMARY_EMAIL'])]
cond1 = cond1[['CONTACT ID', 'FIRM ID']]
cond1


,CONTACT ID,FIRM ID
0,1001,12087
1,1002,10389
2,1003,12087
3,1004,12087
4,1005,10678
...,...,...
995,1996,10890
996,1997,10389
997,1998,10045
998,1999,10450


In [21]:
cond2 = Activity_Details[Activity_Details['ACTIVITY TYPE']=='Email'].sort_values('CONTACT_ID')['CONTACT_ID'].unique()
cond2
cond2 = pd.DataFrame(cond2.T, columns=['CONTACT ID'])
cond2

,CONTACT ID
0,1002
1,1005
2,1006
3,1007
4,1008
...,...
761,1993
762,1994
763,1996
764,1997


In [22]:
cond3 = Advisors_Details[Advisors_Details['UNSUBSCRIBED']!='Blocked']['CONTACT ID']
cond3.info()
cond3 = cond3.reset_index()
cond3.info()

<class 'pandas.core.series.Series'>
Index: 903 entries, 0 to 999
Series name: CONTACT ID
Non-Null Count  Dtype
--------------  -----
903 non-null    int64
dtypes: int64(1)
memory usage: 14.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903 entries, 0 to 902
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   index       903 non-null    int64
 1   CONTACT ID  903 non-null    int64
dtypes: int64(2)
memory usage: 14.2 KB


In [23]:
Initial_Advisors = Advisors_Details[['CONTACT ID', 'FIRM ID']]

In [24]:
Initial_Advisors

,CONTACT ID,FIRM ID
0,1001,12087
1,1002,10389
2,1003,12087
3,1004,12087
4,1005,10678
...,...,...
995,1996,10890
996,1997,10389
997,1998,10045
998,1999,10450


In [25]:
AnswerSet = pd.merge(Initial_Advisors, cond1, on='CONTACT ID', how='inner')
AnswerSet

,CONTACT ID,FIRM ID_x,FIRM ID_y
0,1001,12087,12087
1,1002,10389,10389
2,1003,12087,12087
3,1004,12087,12087
4,1005,10678,10678
...,...,...,...
940,1996,10890,10890
941,1997,10389,10389
942,1998,10045,10045
943,1999,10450,10450


In [26]:
AnswerSet = pd.merge(AnswerSet, cond2, on='CONTACT ID', how='inner')
AnswerSet

,CONTACT ID,FIRM ID_x,FIRM ID_y
0,1002,10389,10389
1,1005,10678,10678
2,1006,10389,10389
3,1007,12087,12087
4,1008,10389,10389
...,...,...,...
724,1992,10089,10089
725,1994,10678,10678
726,1996,10890,10890
727,1997,10389,10389


In [27]:
AnswerSet = pd.merge(AnswerSet, cond3, on='CONTACT ID', how='inner')
AnswerSet

,CONTACT ID,FIRM ID_x,FIRM ID_y,index
0,1005,10678,10678,4
1,1006,10389,10389,5
2,1008,10389,10389,7
3,1010,10678,10678,9
4,1011,10145,10145,10
...,...,...,...,...
649,1992,10089,10089,991
650,1994,10678,10678,993
651,1996,10890,10890,995
652,1997,10389,10389,996


In [30]:
AnswerSet = pd.merge(AnswerSet, target, left_on='CONTACT ID', right_on='CONTACT_ID', how='inner')
AnswerSet

,CONTACT ID,FIRM ID_x,FIRM ID_y,index,CONTACT_ID_x,Percent Change_x,CONTACT_ID_y,Percent Change_y
0,1005,10678,10678,4,1005,-737.801547,1005,-737.801547
1,1008,10389,10389,7,1008,-490.329099,1008,-490.329099
2,1010,10678,10678,9,1010,-2554.881873,1010,-2554.881873
3,1011,10145,10145,10,1011,-2763.787052,1011,-2763.787052
4,1012,13980,13980,11,1012,-3127.904554,1012,-3127.904554
...,...,...,...,...,...,...,...,...
423,1990,10678,10678,989,1990,-1534.590360,1990,-1534.590360
424,1994,10678,10678,993,1994,-2450.879267,1994,-2450.879267
425,1996,10890,10890,995,1996,-80.815245,1996,-80.815245
426,1997,10389,10389,996,1997,-1296.875338,1997,-1296.875338
